In [ ]:
import trimesh
import numpy as np
import pyrender
import pandas as pd
import json

In [ ]:
with open('classes.json')  as file:
    data = json.load(file)

df = pd.json_normalize(data)
print(df)


   classes.no action  classes.inserting vertical short rods  classes.mistakes  \
0                  0                                      1                 2   

   classes.inserting elbows  classes.inserting horizontal short rods  \
0                         3                                        4   

   classes.inserting horizontal long rods  classes.flip  
0                                       5             6  


##Function for creating the 3D model using trimesh instances and Pyrender


In [ ]:
import pandas as pd
import numpy as np
import trimesh
import pyrender

path = "actionsequences.csv"
df = pd.read_csv(path, header=None, names=['Activity'])
df['Activity'] = df['Activity'].str.replace('["\']', '', regex=True)

def create_cylinder(length, radius):
    return trimesh.creation.cylinder(radius=radius, height=length)

def create_leg(radius, height):
    return trimesh.creation.cylinder(radius=radius, height=height)

def reconstructed(df):
    # Desk dimensions and other parameters
    desktop_length = 1.2
    desktop_width = 0.6
    desktop_thickness = 0.05
    edge_radius = 0.025
    ball_radius = 0.03
    leg_radius = 0.05
    leg_height = 0.7

    # Create the top surface of the desktop
    top_surface = trimesh.creation.box(extents=(desktop_length, desktop_width, desktop_thickness))
    color = (255, 255, 255, 255)  # Red color in RGBA format (R, G, B, Alpha)
    top_surface.visual.vertex_colors = np.array([color] * len(top_surface.vertices))

    # Calculate translations for edges
    edge1_translation = [0, desktop_width / 2 - edge_radius - desktop_thickness / 2 + 0.05, edge_radius- 0.016]
    edge2_translation = [0, -desktop_width / 2 + edge_radius + desktop_thickness / 2 - 0.05, edge_radius -0.016]
    edge3_translation = [desktop_length / 2 - edge_radius + 0.05, 0, desktop_thickness / 2 + edge_radius - 0.06]  ##
    edge4_translation = [-desktop_length / 2 + edge_radius - 0.05, 0, desktop_thickness / 2 + edge_radius -0.06]

    # Create edges
    edge1 = create_cylinder(desktop_length, edge_radius)
    edge1.apply_translation(edge1_translation)
    edge1.apply_transform(trimesh.transformations.rotation_matrix(np.pi / 2, [0, 1, 0]))

    edge2 = create_cylinder(desktop_length, edge_radius)
    edge2.apply_translation(edge2_translation)
    edge2.apply_transform(trimesh.transformations.rotation_matrix(np.pi / 2, [0, 1, 0]))

    edge3 = create_cylinder(desktop_width+0.03, edge_radius)
    edge3.apply_translation(edge3_translation)
    edge3.apply_transform(trimesh.transformations.rotation_matrix(np.pi / 2, [1, 0, 0]))

    edge4 = create_cylinder(desktop_width+0.03, edge_radius)
    edge4.apply_translation(edge4_translation)
    edge4.apply_transform(trimesh.transformations.rotation_matrix(np.pi / 2, [1, 0, 0]))

    # Create balls at the corners
    ball1_translation = [desktop_length / 2 - ball_radius+0.06, desktop_width / 2 - ball_radius+0.02, desktop_thickness / 2]
    ball2_translation = [desktop_length / 2 - ball_radius+0.06, -desktop_width / 2 + ball_radius-0.02, desktop_thickness / 2]
    ball3_translation = [-desktop_length / 2 + ball_radius-0.06, desktop_width / 2 - ball_radius+0.02, desktop_thickness / 2]
    ball4_translation = [-desktop_length / 2 + ball_radius-0.06, -desktop_width / 2 + ball_radius-0.02, desktop_thickness / 2]

    ball1 = trimesh.creation.icosphere(radius=ball_radius)
    ball1.apply_translation(ball1_translation)

    ball2 = trimesh.creation.icosphere(radius=ball_radius)
    ball2.apply_translation(ball2_translation)

    ball3 = trimesh.creation.icosphere(radius=ball_radius)
    ball3.apply_translation(ball3_translation)

    ball4 = trimesh.creation.icosphere(radius=ball_radius)
    ball4.apply_translation(ball4_translation)

    # Create the legs using the provided create_leg function
    leg1 = create_leg(leg_radius, leg_height)
    leg1.apply_translation([desktop_length / 2 - leg_radius, desktop_width / 2 - leg_radius, -desktop_thickness - 0.3])

    leg2 = create_leg(leg_radius, leg_height)
    leg2.apply_translation([desktop_length / 2 - leg_radius, -desktop_width / 2 + leg_radius, -desktop_thickness - 0.3])

    leg3 = create_leg(leg_radius, leg_height)
    leg3.apply_translation([-desktop_length / 2 + leg_radius, desktop_width / 2 - leg_radius, -desktop_thickness - 0.3])

    leg4 = create_leg(leg_radius, leg_height)
    leg4.apply_translation([-desktop_length / 2 + leg_radius, -desktop_width / 2 + leg_radius, -desktop_thickness - 0.3])

    # Start combining parts based on the input csv file
    desktop = top_surface
    ihlr, ie, ihsr, ivsr = 1, 1, 1, 1

    for index, row in df.iterrows():
        activity = row['Activity']

        if activity == 'inserting horizontal long rods':
            if ihlr == 1:
                desktop += edge1
                ihlr += 1
            elif ihlr == 2:
                desktop += edge2
                ihlr += 1

        elif activity == 'inserting elbows':
            if ie == 1:
                desktop += ball1
                ie += 1
            elif ie == 2:
                desktop += ball2
                ie += 1
            elif ie == 3:
                desktop += ball3
                ie += 1
            elif ie == 4:
                desktop += ball4
                ie += 1

        elif activity == 'inserting horizontal short rods':
            if ihsr == 1:
                desktop += edge3
                ihsr += 1
            elif ihsr == 2:
                desktop += edge4
                ihsr += 1

        elif activity == 'inserting vertical short rods':
            if ivsr == 1:
                desktop += leg1
                ivsr += 1
            elif ivsr == 2:
                desktop += leg2
                ivsr += 1
            elif ivsr == 3:
                desktop += leg3
                ivsr += 1
            elif ivsr == 4:
                desktop += leg4
                ivsr += 1

    # Convert to a format suitable for PyRender with per-vertex colors
        mesh = pyrender.Mesh.from_trimesh(desktop, smooth=False)


        scene = pyrender.Scene()
        scene.add(mesh)

    # Render the scene
        viewer = pyrender.Viewer(scene, use_raymond_lighting=True)


reconstructed(df)
